In [43]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [45]:
design.chips['C_chip'].size['size_x']='3mm'
design.chips['C_chip'].size['size_y']='3mm'
design.chips['Q_chip'].size['size_x']='2mm'
design.chips['Q_chip'].size['size_y']='2mm'
design.chips['Q_chip'].size['center_z']='8um'

In [46]:
design.get_chip_z('Q_chip')

'8um'

In [47]:

cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-300um',
                                            pos_y='-500um',
                                            chip='Q_chip',
                                            pad_width='370um',
                                            pad_height='100um',
                                            pad_gap='80um',
                                            pocket_width='680um',
                                            pocket_height='680um'))]
                                            
qubits=q0
q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = StraightUpCoupler(design, "c0", options=dict(pos_x=f"{q0x-120}um",
                                              pos_y=f"{q0y+90}um",
                                              pad_width="110um",
                                              pad_height="100um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))
design.rebuild()
gui.rebuild()
gui.autoscale()

In [48]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-700um',
                                         pos_y = '800um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '700um',
                                         pos_y = '800um',
                                         orientation='180',
                                         lead_length = '30um'))                                        

In [49]:
stg=ShortToGround(design, 'stg', options=Dict(chip="C_chip",
                                                pos_x='0.65mm',
                                                pos_y='-0.2mm',
                                                orientation='270'))

In [50]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [51]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

# Resonator length
3.85mm

In [52]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [53]:
resonator_length_4=[]
a=find_resonator_length(7.5,10,6,4)
a=a.strip('mm')
a=float(a)-0.25 +0.5-0.23-0.5# subtract coupler length, for +0.5Ghz, coupler length
resonator_length_4.append(a)
print(resonator_length_4)


[3.5775622473488387]


In [54]:
jogs1=OrderedDict()
jogs1[0]=['R','300um']
jogs1[1]=['R', '300um']
jogs1[2]=['L','700um']
jogs1[3]=['R','200um']
jogs1[4]=['R','670um']
jogs1[5]=['L','200um']
jogs1[6]=['L','740um']


In [55]:
rr0=connect('rr0', 'c0', 'in','stg', 'short',
            resonator_length_4[0],jogs1,jogs, 
            '100um','0um', '944um', '100um')
            

# EPR Analysis

In [56]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [57]:
hfss.start()

INFO 03:38PM [connect_project]: Connecting to Ansys Desktop API...
INFO 03:38PM [load_ansys_project]: 	Opened Ansys App
INFO 03:38PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 03:38PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Flipchip Bump
INFO 03:38PM [connect_design]: 	Opened active design
	Design:    flip7um_V6 [Solution type: Eigenmode]
INFO 03:38PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 03:38PM [connect]: 	Connected to project "Flipchip Bump" and design "flip7um_V6" 😀 



True

In [58]:
hfss.activate_ansys_design("flip8um_V6", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','stg','rr0','fl0'],[],
                    box_plus_buffer=False)

03:38PM 54s WARNING [activate_ansys_design]: The design_name=flip8um_V6 was not in active project.  Designs in active project are: 
['flip7um_V6', 'flip6um_V6'].  A new design will be added to the project.  
INFO 03:38PM [connect_design]: 	Opened active design
	Design:    flip8um_V6 [Solution type: Eigenmode]
WARNING 03:38PM [connect_setup]: 	No design setup detected.
WARNING 03:38PM [connect_setup]: 	Creating eigenmode default setup.
INFO 03:38PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [59]:
hfss.close()


Warning! 3 COM references still alive
Ansys will likely refuse to shut down


True

# LOM Analysis

In [19]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [20]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.67',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.67',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [22]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 02:59PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpdd1ba5h5.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5670000000, Maxwell, 1, False
INFO 02:59PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpwhea6y2k.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5670000000, Maxwell, 1, False
INFO 02:59PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmph9wbr48a.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5670000000, Maxwell, 2, False
INFO 02:59PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpy0yjqt32.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5670000000, Maxwell, 3, False
INFO 02:59PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpt7h94wp7.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5670000000, Maxwell, 4, False
INFO 02:59

,pad_bot_Q0,pad_top_Q0,quad_coupler_c0,SignalNet1
pad_bot_Q0,122.110000,-12.420,-0.048589,-108.220
pad_top_Q0,-12.420000,122.180,-14.206000,-94.094
quad_coupler_c0,-0.048589,-14.206,93.401000,-78.953
SignalNet1,-108.220000,-94.094,-78.953000,343.900


In [23]:
c1.setup.junctions = Dict({'Lj': 12, 'Cj': 2})
c1.setup.freq_readout = 6.76
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.520951 [GHz]
EC 314.631248 [MHz]
EJ 13.616300 [GHz]
alpha -366.264469 [MHz]
dispersion 41.028296 [KHz]
Lq 11.995161 [nH]
Cq 61.564858 [fF]
T1 70.111154 [us]

**Coupling Properties**

tCqbus1 -5.261145 [fF]
gbus1_in_MHz -79.858215 [MHz]
χ_bus1 -2.380563 [MHz]
1/T1bus1 2270.037403 [Hz]
T1bus1 70.111154 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,6.424969,436.243394,13.6163,-527.415424,601.859157,[-43.28331123891727],[-6.851057384247964],6.851057,43.283311
2,5.778586,346.941812,13.6163,-407.983204,96.25979,[-67.40599473872115],[-2.743258031922683],2.743258,67.405995
3,5.616148,326.357886,13.6163,-381.317445,56.750143,[-80.10283688104438],[-2.8379116853049142],2.837912,80.102837
4,5.599752,324.320521,13.6163,-378.695001,53.709335,[-84.22152001453972],[-3.0450739558729296],3.045074,84.221520
5,5.57591,321.371063,13.6163,-374.903884,49.5468,[-84.03791719345995],[-2.90449890011342],2.904499,84.037917
6,5.564814,320.003649,13.6163,-373.148408,47.709635,[-80.10729993390574],[-2.5874570075365773],2.587457,80.107300
7,5.559978,319.408731,13.6163,-372.38508,46.928141,[-80.03593236208269],[-2.560765687251143],2.560766,80.035932
8,5.546977,317.812605,13.6163,-370.338387,44.883646,[-78.29606546489646],[-2.394973312269079],2.394973,78.296065
9,5.530936,315.849633,13.6163,-367.823819,42.470975,[-79.44014790973854],[-2.3970609582897344],2.397061,79.440148
10,5.520951,314.631248,13.6163,-366.264469,41.028296,[-79.85821540074018],[-2.380563322557715],2.380563,79.858215


In [42]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
